In [1]:
import pandas as pd
import rdkit
import scipy.sparse
import os

In [2]:
## file containing image features indexed with the input_compound_id
features_file = './images_files/input_hofmarcher/T_image_features_std.csv'

## Folder containing melloddy tuner output on image compounds
tuner_output_images = './images_files/output_files/image_model'



In [4]:
t5 = pd.read_csv(os.path.join(tuner_output_images, 'mapping_table/T5.csv'))
t5 = t5[['input_compound_id','descriptor_vector_id']]

t10_c = pd.read_csv(os.path.join(tuner_output_images, 'results/T10c_cont.csv'))
t10_c = t10_c [['descriptor_vector_id','cont_descriptor_vector_id']].drop_duplicates()

cmpd_mapping_table = pd.merge(t5, t10_c, on='descriptor_vector_id', how='inner')
cmpd_mapping_table

,input_compound_id,descriptor_vector_id,cont_descriptor_vector_id
0,845253,3607,3479
1,298290,2447,2344
2,922657,4652,4464
3,62817,1122,1084
4,1001579,1481,1433
...,...,...,...
10062,773366,8753,8410
10063,989091,4554,4368
10064,26590,4650,4462
10065,736500,389,360


In [5]:
T_image_features = pd.read_csv(features_file)


In [6]:
x_features = pd.merge(T_image_features, cmpd_mapping_table, on='input_compound_id', how='right')
x_features = x_features.drop(['input_compound_id','descriptor_vector_id'], axis=1)
x_features = x_features.groupby('cont_descriptor_vector_id').agg('mean')

In [7]:
assert len(x_features)==len(t10_c)

In [9]:
cls_T11_x_features = scipy.sparse.csr_matrix(x_features.values)


In [10]:
scipy.sparse.save_npz(file=os.path.join(tuner_output_images, 'matrices/cls/cls_T11_x_features.npz'), matrix=cls_T11_x_features)
